In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import nn_vis
import matplotlib.pyplot as plt
from torchviz import make_dot
from IPython.display import Image, display

# Model Definition


In [ ]:
class FeedForward(nn.Module):
    def __init__(self):
        super().__init__()
        # Layers
        self.layer1 = nn.Linear(1, 10)
        self.layer2 = nn.Linear(10, 10)
        self.layer3 = nn.Linear(10, 10)
        self.layer4 = nn.Linear(10, 1)

    def forward(self, x):
        # Forward pass, layer by layer
        x = self.layer1(x)
        x = torch.tanh(x)
        x = self.layer2(x)
        x = torch.tanh(x)
        x = self.layer3(x)
        x = torch.tanh(x)
        x = self.layer4(x)
        return x

In [ ]:
# Create a model instance
model = FeedForward()

In [ ]:
nn_vis.visualize_neural_network(model)

In [ ]:
def display_model(model):
    sample_input = torch.randn(1)
    output = model(sample_input)
    dot = make_dot(output, params=dict(model.named_parameters()))
    dot.render("simple_ffn_graph", format="png")
    display(Image(filename="simple_ffn_graph.png"))


display_model(model)

# Sampling a Function


In [ ]:
# Generate sine wave data
x = np.linspace(0, 2 * np.pi, 1000)
y = np.sin(x)

# Display the sine wave
plt.figure(figsize=(8, 4))
plt.plot(x, y, label="Sine Wave")
plt.title("Sine Wave")
plt.xlabel("x")
plt.ylabel("sin(x)")
plt.legend()
plt.show()

In [ ]:
N_SAMPLES = 10

# Sample 100 points from the sine wave data
sample_indices = np.random.choice(len(x), size=N_SAMPLES, replace=False)
x_sample = x[sample_indices]
y_sample = y[sample_indices]

# Plot the sampled points on top of the sine wave
plt.figure(figsize=(8, 4))
plt.plot(x, y, label="Sine Wave")
plt.scatter(x_sample, y_sample, color="red", label="Sampled Points")
plt.title("Sine Wave with Sampled Points")
plt.xlabel("x")
plt.ylabel("sin(x)")
plt.legend()
plt.show()

In [ ]:
# Convert sampled data to tensors
x_train = torch.tensor(x_sample, dtype=torch.float32).unsqueeze(1)
y_train = torch.tensor(y_sample, dtype=torch.float32).unsqueeze(1)

In [ ]:
print(x_train)

In [ ]:
print(y_train)

# Training the Model


In [ ]:
model = FeedForward()

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
EPOCHS = 700
losses = []

for epoch in range(EPOCHS):
    # Prep for next loop
    model.train()
    optimizer.zero_grad()

    # Compute predictions at sampled points
    outputs = model(x_train)

    # How far off are we?
    loss = criterion(outputs, y_train)

    # Correct weights
    loss.backward()
    optimizer.step()

    # Store losses for plotting
    losses.append(loss.item())

    print(f"Epoch {epoch + 1}/{EPOCHS}, Loss: {loss.item():.6f}")

# Reviewing Results


In [ ]:
# Plot loss curve
plt.figure(figsize=(8, 4))
plt.plot(losses)
plt.title("Training Loss Curve")
plt.xlabel("Epoch")
plt.ylabel("MSE Loss")
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Generate inputs from 0 to 2pi
x_test = np.linspace(0, 2 * np.pi, 100)
x_test_tensor = torch.tensor(x_test, dtype=torch.float32).unsqueeze(1)

# Get model predictions
model.eval()
with torch.no_grad():
    y_pred = model(x_test_tensor).squeeze().numpy()

# Display the model outputs with matplotlib and plot sine wave for comparison
plt.figure(figsize=(8, 4))
plt.plot(x_test, y_pred, label="Model Output")
plt.plot(x_test, np.sin(x_test), "k:", label="Sine Wave (dotted)")
plt.title("Model Predictions")
plt.xlabel("x")
plt.ylabel("y_pred")
plt.legend()
plt.show()